In [1]:
# Riesenie Kristian Toldy, Predmet: Vyhladavanie Informacii FIIT STU

In [2]:
import xml.sax
import re

# Regex pattern zakladne linky
pattern = "\[\[(.*)\]\]"

# Streamovy parser XML dat
class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ""
        self.title = ""
        self.id = ""
        self.text = ""
        
        
    def startElement(self, tag, attributes):
        self.CurrentData = tag
            
    def endElement(self, tag):
        #Vyskušame vždy pridať novú pagu do nášho dictionary (HashMapa)
        if self.CurrentData == "title":
            try:
                if dict[self.title]:
                    print("Duplicate found:", self.title)
            except KeyError:
                dict[self.title] = len(dict)
            list.append(self.title)
#        elif self.CurrentData == "text":
#            print("text: ", self.text)
        elif self.CurrentData == "text":
            if(re.search(pattern, self.text)):
                links.append(self.title)
                links.append(re.search(pattern, self.text).group(1))
                #print("Link: ", re.search(pattern, self.text).group(1))
                
            
        self.CurrentData = ""
        
    def characters(self, content):
        if self.CurrentData == "title":
            self.title = content
        elif self.CurrentData == "id":
            self.id = content
        elif self.CurrentData == "text":
            self.text = content
          
        
if (__name__ == "__main__"):
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    Handler = XMLHandler()
    parser.setContentHandler(Handler)
    
    
    list = []  # < - Obsahuje zoznam pagov
    links = [] # < - Obsahuje zoznam linkov vo formate x, x+1 kde je title page1 ukazujucu na page2
    dict = {} # < - Hash mapa na rýchle nájdenie IDcka stranky podla title

    # Sem vlozte vas XML subor
    parser.parse("C:/Users/krist/Desktop/Fiit/Vyhladavanie informacii/Testovanie/wiki.xml")
    
    

Duplicate found:  City Line
Duplicate found: B
Duplicate found: Y
Duplicate found: M
Duplicate found: T
Duplicate found:  Koch
Duplicate found:  Conquer
Duplicate found: W
Duplicate found: "
Duplicate found: "
Duplicate found: "
Duplicate found:  The Stooges
Duplicate found: Boca Juniors
Duplicate found: M
Duplicate found:  Bass
Duplicate found:  Coach
Duplicate found:  Bass
Duplicate found:  Bass
Duplicate found: 1966
Duplicate found: "
Duplicate found:  Breeze
Duplicate found:  Status
Duplicate found:  Koch
Duplicate found:  Status
Duplicate found: "
Duplicate found:  the Banshees
Duplicate found:  the Banshees
Duplicate found: "
Duplicate found: "
Duplicate found: T
Duplicate found:  Klement
Duplicate found: "
Duplicate found:  Bassline Smith
Duplicate found: B
Duplicate found: B
Duplicate found:  Futurebound
Duplicate found:  Evans
Duplicate found:  Status
Duplicate found:  Spoon
Duplicate found:  Krooked
Duplicate found:  Co Beschläge
Duplicate found:  KOS
Duplicate found:  Years


In [4]:
# Vygenerovanie numpy arrayu linkov kde menime link title na ID
import numpy as np
list2 = np.empty((0,2), int)
for x in range(0, len(links),2):
    try:
        list2 = np.append(list2, np.array([[dict[links[x]], dict[links[x+1]]]]), axis=0)
    except KeyError:
        print("Link nenajdeny:", dict[links[x]]) # < - Ak pagu nepozname na ktorú ukazuje dá nám chybu a v ktorej page sa chyba vyskytla

print(list2)

Link nenajdeny: 21
Link nenajdeny: 22
Link nenajdeny: 55
Link nenajdeny: 66
Link nenajdeny: 69
Link nenajdeny: 71
Link nenajdeny: 72
Link nenajdeny: 75
Link nenajdeny: 77
Link nenajdeny: 79
Link nenajdeny: 80
Link nenajdeny: 81
Link nenajdeny: 82
Link nenajdeny: 88
Link nenajdeny: 89
Link nenajdeny: 90
Link nenajdeny: 91
Link nenajdeny: 92
Link nenajdeny: 98
Link nenajdeny: 99
Link nenajdeny: 104
Link nenajdeny: 107
Link nenajdeny: 115
Link nenajdeny: 118
Link nenajdeny: 119
Link nenajdeny: 120
Link nenajdeny: 126
Link nenajdeny: 138
Link nenajdeny: 157
Link nenajdeny: 172
Link nenajdeny: 179
Link nenajdeny: 180
Link nenajdeny: 183
Link nenajdeny: 184
Link nenajdeny: 185
Link nenajdeny: 186
Link nenajdeny: 187
Link nenajdeny: 189
Link nenajdeny: 191
Link nenajdeny: 197
Link nenajdeny: 201
Link nenajdeny: 211
Link nenajdeny: 213
Link nenajdeny: 225
Link nenajdeny: 227
Link nenajdeny: 229
Link nenajdeny: 242
Link nenajdeny: 246
Link nenajdeny: 248
Link nenajdeny: 255
Link nenajdeny: 262


Link nenajdeny: 2812
Link nenajdeny: 2814
Link nenajdeny: 2816
Link nenajdeny: 2819
Link nenajdeny: 2822
Link nenajdeny: 2839
Link nenajdeny: 2840
Link nenajdeny: 2841
Link nenajdeny: 2842
Link nenajdeny: 2843
Link nenajdeny: 2844
Link nenajdeny: 2845
Link nenajdeny: 2846
Link nenajdeny: 2847
Link nenajdeny: 2848
Link nenajdeny: 2849
Link nenajdeny: 2850
Link nenajdeny: 2851
Link nenajdeny: 2852
Link nenajdeny: 2853
Link nenajdeny: 2854
Link nenajdeny: 2856
Link nenajdeny: 2858
Link nenajdeny: 2859
Link nenajdeny: 2860
Link nenajdeny: 2861
Link nenajdeny: 2862
Link nenajdeny: 2863
Link nenajdeny: 2864
Link nenajdeny: 2865
Link nenajdeny: 2866
Link nenajdeny: 2873
Link nenajdeny: 2875
Link nenajdeny: 2877
Link nenajdeny: 2878
Link nenajdeny: 2879
Link nenajdeny: 2880
Link nenajdeny: 2881
Link nenajdeny: 2882
Link nenajdeny: 2883
Link nenajdeny: 2884
Link nenajdeny: 2885
Link nenajdeny: 2886
Link nenajdeny: 2887
Link nenajdeny: 2888
Link nenajdeny: 2890
Link nenajdeny: 2891
Link nenajden

Link nenajdeny: 6230
Link nenajdeny: 6234
Link nenajdeny: 6238
Link nenajdeny: 6247
Link nenajdeny: 6248
Link nenajdeny: 6252
Link nenajdeny: 6254
Link nenajdeny: 6255
Link nenajdeny: 6256
Link nenajdeny: 6258
Link nenajdeny: 6259
Link nenajdeny: 6260
Link nenajdeny: 6261
Link nenajdeny: 6262
Link nenajdeny: 6263
Link nenajdeny: 6264
Link nenajdeny: 6267
Link nenajdeny: 6268
Link nenajdeny: 6270
Link nenajdeny: 6275
Link nenajdeny: 6276
Link nenajdeny: 6277
Link nenajdeny: 6278
Link nenajdeny: 6279
Link nenajdeny: 6293
Link nenajdeny: 6303
Link nenajdeny: 6304
Link nenajdeny: 6305
Link nenajdeny: 6306
Link nenajdeny: 6314
Link nenajdeny: 6315
Link nenajdeny: 6316
Link nenajdeny: 6317
Link nenajdeny: 6318
Link nenajdeny: 6319
Link nenajdeny: 6320
Link nenajdeny: 6321
Link nenajdeny: 6322
Link nenajdeny: 6323
Link nenajdeny: 6329
Link nenajdeny: 6330
Link nenajdeny: 6332
Link nenajdeny: 6337
Link nenajdeny: 6338
Link nenajdeny: 6352
Link nenajdeny: 6353
Link nenajdeny: 6356
Link nenajden

Link nenajdeny: 7583
Link nenajdeny: 7584
Link nenajdeny: 7585
Link nenajdeny: 7586
Link nenajdeny: 7587
Link nenajdeny: 7588
Link nenajdeny: 7589
Link nenajdeny: 7590
Link nenajdeny: 7591
Link nenajdeny: 7592
Link nenajdeny: 7593
Link nenajdeny: 7594
Link nenajdeny: 7595
Link nenajdeny: 7596
Link nenajdeny: 7597
Link nenajdeny: 7598
Link nenajdeny: 7599
Link nenajdeny: 7600
Link nenajdeny: 7601
Link nenajdeny: 7602
Link nenajdeny: 7603
Link nenajdeny: 7604
Link nenajdeny: 7605
Link nenajdeny: 7606
Link nenajdeny: 7607
Link nenajdeny: 7608
Link nenajdeny: 7609
Link nenajdeny: 7610
Link nenajdeny: 7611
Link nenajdeny: 7612
Link nenajdeny: 7613
Link nenajdeny: 7614
Link nenajdeny: 7615
Link nenajdeny: 7616
Link nenajdeny: 7617
Link nenajdeny: 7618
Link nenajdeny: 7619
Link nenajdeny: 7620
Link nenajdeny: 7621
Link nenajdeny: 7622
Link nenajdeny: 7623
Link nenajdeny: 7624
Link nenajdeny: 7625
Link nenajdeny: 7626
Link nenajdeny: 7627
Link nenajdeny: 7628
Link nenajdeny: 7629
Link nenajden

Link nenajdeny: 9870
Link nenajdeny: 9871
Link nenajdeny: 9872
Link nenajdeny: 9873
Link nenajdeny: 9874
Link nenajdeny: 9875
Link nenajdeny: 9876
Link nenajdeny: 9877
Link nenajdeny: 9878
Link nenajdeny: 9879
Link nenajdeny: 9880
Link nenajdeny: 9881
Link nenajdeny: 9882
Link nenajdeny: 9883
Link nenajdeny: 9884
Link nenajdeny: 9885
Link nenajdeny: 9886
Link nenajdeny: 9887
Link nenajdeny: 9888
Link nenajdeny: 9889
Link nenajdeny: 9890
Link nenajdeny: 9891
Link nenajdeny: 9892
Link nenajdeny: 9893
Link nenajdeny: 9894
Link nenajdeny: 9895
Link nenajdeny: 9896
Link nenajdeny: 9897
Link nenajdeny: 9898
Link nenajdeny: 9899
Link nenajdeny: 9900
Link nenajdeny: 9901
Link nenajdeny: 9902
Link nenajdeny: 9903
Link nenajdeny: 9904
Link nenajdeny: 9905
Link nenajdeny: 9906
Link nenajdeny: 9907
Link nenajdeny: 9908
Link nenajdeny: 9909
Link nenajdeny: 9910
Link nenajdeny: 9913
Link nenajdeny: 9972
Link nenajdeny: 9979
Link nenajdeny: 9980
Link nenajdeny: 9982
Link nenajdeny: 9983
Link nenajden

...

Link nenajdeny: 410327
Link nenajdeny: 410328
Link nenajdeny: 410329
Link nenajdeny: 410330
Link nenajdeny: 410331
Link nenajdeny: 410332
Link nenajdeny: 410333
Link nenajdeny: 410334
Link nenajdeny: 410335
Link nenajdeny: 410336
Link nenajdeny: 410337
Link nenajdeny: 410338
Link nenajdeny: 410339
Link nenajdeny: 410340
Link nenajdeny: 410341
Link nenajdeny: 410342
Link nenajdeny: 410343
Link nenajdeny: 410345
Link nenajdeny: 410346
Link nenajdeny: 410347
Link nenajdeny: 410348
Link nenajdeny: 410349
Link nenajdeny: 410350
Link nenajdeny: 410351
Link nenajdeny: 410352
Link nenajdeny: 410353
Link nenajdeny: 410354
Link nenajdeny: 410355
Link nenajdeny: 410356
Link nenajdeny: 410357
Link nenajdeny: 410358
Link nenajdeny: 410359
Link nenajdeny: 410361
Link nenajdeny: 410365
Link nenajdeny: 410367
Link nenajdeny: 410369
Link nenajdeny: 410370
Link nenajdeny: 410371
Link nenajdeny: 410372
Link nenajdeny: 410376
Link nenajdeny: 410378
Link nenajdeny: 410380
Link nenajdeny: 410381
Link nenajd

Link nenajdeny: 411688
Link nenajdeny: 411689
Link nenajdeny: 411690
Link nenajdeny: 411691
Link nenajdeny: 411692
Link nenajdeny: 411696
Link nenajdeny: 411697
Link nenajdeny: 411700
Link nenajdeny: 411701
Link nenajdeny: 411702
Link nenajdeny: 411703
Link nenajdeny: 411712
Link nenajdeny: 411716
Link nenajdeny: 411727
Link nenajdeny: 411745
Link nenajdeny: 411788
Link nenajdeny: 411789
Link nenajdeny: 411791
Link nenajdeny: 411802
Link nenajdeny: 411806
Link nenajdeny: 411807
Link nenajdeny: 411816
Link nenajdeny: 411824
Link nenajdeny: 411825
Link nenajdeny: 411826
Link nenajdeny: 411830
Link nenajdeny: 411831
Link nenajdeny: 411834
Link nenajdeny: 411835
Link nenajdeny: 411836
Link nenajdeny: 411837
Link nenajdeny: 411845
Link nenajdeny: 411847
Link nenajdeny: 411851
Link nenajdeny: 411864
Link nenajdeny: 411865
Link nenajdeny: 411870
Link nenajdeny: 411872
Link nenajdeny: 411873
Link nenajdeny: 411874
Link nenajdeny: 411877
Link nenajdeny: 411878
Link nenajdeny: 411879
Link nenajd

Link nenajdeny: 413652
Link nenajdeny: 413658
Link nenajdeny: 413671
Link nenajdeny: 413672
Link nenajdeny: 413673
Link nenajdeny: 413694
Link nenajdeny: 413695
Link nenajdeny: 413696
Link nenajdeny: 413698
Link nenajdeny: 413699
Link nenajdeny: 413700
Link nenajdeny: 413701
Link nenajdeny: 413708
Link nenajdeny: 413709
Link nenajdeny: 413710
Link nenajdeny: 413711
Link nenajdeny: 413714
Link nenajdeny: 413716
Link nenajdeny: 413717
Link nenajdeny: 413718
Link nenajdeny: 413719
Link nenajdeny: 413720
Link nenajdeny: 413725
Link nenajdeny: 413726
Link nenajdeny: 413730
Link nenajdeny: 413732
Link nenajdeny: 413742
Link nenajdeny: 413743
Link nenajdeny: 413744
Link nenajdeny: 413746
Link nenajdeny: 413747
Link nenajdeny: 413751
Link nenajdeny: 413757
Link nenajdeny: 413781
Link nenajdeny: 413782
Link nenajdeny: 413800
Link nenajdeny: 413801
Link nenajdeny: 413809
Link nenajdeny: 413810
Link nenajdeny: 413811
Link nenajdeny: 413813
Link nenajdeny: 413814
Link nenajdeny: 413845
Link nenajd

Link nenajdeny: 415932
Link nenajdeny: 415947
Link nenajdeny: 415963
Link nenajdeny: 415967
Link nenajdeny: 415970
Link nenajdeny: 415972
Link nenajdeny: 415973
Link nenajdeny: 415979
Link nenajdeny: 415982
Link nenajdeny: 415983
Link nenajdeny: 415993
Link nenajdeny: 415994
Link nenajdeny: 416017
Link nenajdeny: 416022
Link nenajdeny: 416023
Link nenajdeny: 416026
Link nenajdeny: 416028
Link nenajdeny: 416029
Link nenajdeny: 416031
Link nenajdeny: 416032
Link nenajdeny: 416033
Link nenajdeny: 416034
Link nenajdeny: 416035
Link nenajdeny: 416038
Link nenajdeny: 416042
Link nenajdeny: 416043
Link nenajdeny: 416044
Link nenajdeny: 416045
Link nenajdeny: 416049
Link nenajdeny: 416053
Link nenajdeny: 416054
Link nenajdeny: 416055
Link nenajdeny: 416056
Link nenajdeny: 416059
Link nenajdeny: 416060
Link nenajdeny: 416061
Link nenajdeny: 416063
Link nenajdeny: 416064
Link nenajdeny: 416065
Link nenajdeny: 416066
Link nenajdeny: 416067
Link nenajdeny: 416069
Link nenajdeny: 416071
Link nenajd

Link nenajdeny: 417633
Link nenajdeny: 417636
Link nenajdeny: 417637
Link nenajdeny: 417639
Link nenajdeny: 417643
Link nenajdeny: 417644
Link nenajdeny: 417645
Link nenajdeny: 417646
Link nenajdeny: 417648
Link nenajdeny: 417649
Link nenajdeny: 417651
Link nenajdeny: 417652
Link nenajdeny: 417653
Link nenajdeny: 417654
Link nenajdeny: 417657
Link nenajdeny: 417658
Link nenajdeny: 417659
Link nenajdeny: 417662
Link nenajdeny: 417664
Link nenajdeny: 417666
Link nenajdeny: 417667
Link nenajdeny: 417668
Link nenajdeny: 417670
Link nenajdeny: 417672
Link nenajdeny: 417673
Link nenajdeny: 417674
Link nenajdeny: 417675
Link nenajdeny: 417676
Link nenajdeny: 417678
Link nenajdeny: 417679
Link nenajdeny: 417681
Link nenajdeny: 417682
Link nenajdeny: 417683
Link nenajdeny: 417684
Link nenajdeny: 417685
Link nenajdeny: 417686
Link nenajdeny: 417687
Link nenajdeny: 417688
Link nenajdeny: 417689
Link nenajdeny: 417691
Link nenajdeny: 417692
Link nenajdeny: 417694
Link nenajdeny: 417695
Link nenajd

In [8]:
# Vypocet PageRanku
import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg

# Vstupy:
# G - CSR graph linkov
# D - Damping factor
# max_iter - maximalny pocet iteracii
# tolerancy -  Tolerianca ked uz convergujeme aby ukoncilo vykonavanie
def pagerank(G, D, max_iter, tolerancy=1e-05):
   

    # n pocet prvkov v grafe
    n, _ = G.shape
    # Spocita pocet outgoing linkov z pagy v nasej CSR matici (pre kazdy riadok)
    outgoing = sp.asarray(G.sum(axis=1)).reshape(-1)
    # indexes - indexy ktore su nenulove, teda maju outgoing link
    indexes = outgoing.nonzero()[0]
    # Nova matica kde vydelim hodnotu outgoing linkov poctom linkov (cim viac tim mensi zasah ma stranka)
    impact_csr = sprs.csr_matrix((1 / outgoing[indexes], (indexes, indexes)), shape=(n, n))

    # Vytvorenie jednotkoveho vektora
    ones = sp.ones(n) 
    ones = ones.reshape(n, 1)
    
    # a[np.newaxis, :] - > Pridanie dimenzie
    # Riesenie dangling nodes (nemaju ziadne outgoing, 
    # inicializujeme tie co nemaju ziadne outgoing linky na (1-dump)/n a tie co maju maju dump/n
    # Teoria - > ked neexistuje outgoing, tak mame nahodnu sancu otvorit akukolvek inu stranku (- dump)
    jumps = (((1 - D) * (outgoing != 0) + (outgoing == 0)) / n)[sp.newaxis, :]
    
    # A.T otoci smer ukazovania, pojdeme po linkoch namiesto po pagoch
    # W vypocita silu hrany (V podstate )
    W = D * G.T @ impact_csr
    
    # X pole page rankov (Na zaciatok vsetky jedna)
    x = ones
    # oldX v predchadzajucej iteracii pagerank, na zaciatok same nuly
    oldx = sp.zeros((n, 1))
    
    iteration = 0

    while sp.linalg.norm(x - oldx) > tolerancy:
        oldx = x
        # Vypocet noveho pageranku pre vsetky page 
        x = W @ x + ones @ (jumps @ x)
        iteration += 1
        if iteration >= max_iter:
            break
    x = x / sum(x)

    return x.reshape(-1)

In [9]:
from scipy import sparse

# List jednotiek, su to nase vahy linkov, vsetky maju rovnaku silu
weights = [1] * len(list2)
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]
# Shape musí obsahovať aj stranky bez odkazov, Len(list_stranok)
G = sparse.csr_matrix((weights, (list2[:,0], list2[:,1])), shape=(len(list), len(list)))
rank=pagerank(G, D=0.85, max_iter=10)
rank

array([3.69224055e-06, 1.03969957e-06, 1.03969957e-06, ...,
       1.03969957e-06, 1.03969957e-06, 1.03969957e-06])

In [10]:
len(list)
# Statistika Slovenskej wiki: https://sk.wikipedia.org/wiki/%C5%A0peci%C3%A1lne:%C5%A0tatistika
# Pocet pagov vo wiki:

418027

In [14]:
# Hladanie najsilnejsieho pagu
iterator = 0
m = max(rank)
for x in rank:
    iterator = iterator + 1
    if x == m:
        print(iterator)

3256


In [15]:
list[3256]

'Všeobecná úverová banka'

In [17]:
#Hodnota page ranku najsilnejsej stranky 
rank[3256]*len(list)

1.173844992091675

In [18]:
# Na mensej vzorke dat:
# 10 pageov,
# 2 validne linky na ine page, 1 bez landingu
list = []
links = []
dict = {}
parser.parse("C:/Users/krist/Desktop/Fiit/Vyhladavanie informacii/Testovanie/wiki2.xml")
print(links)

['MediaWiki:Developertitle', 'MediaWiki:Newmessages', 'MediaWiki:Developertext', 'MediaWiki:Commentedit', 'MediaWiki:Searchresulttext', 'Pomoc:Vyhľadávanie']


In [19]:
# Vygenerovanie numpy arrayu linkov kde menime link title na ID
import numpy as np
list2 = np.empty((0,2), int)
for x in range(0, len(links),2):
    try:
        list2 = np.append(list2, np.array([[dict[links[x]], dict[links[x+1]]]]), axis=0)
    except KeyError:
        print("Link nenajdeny:", dict[links[x]]) # < - Ak pagu nepozname na ktorú ukazuje dá nám chybu a v ktorej page sa chyba vyskytla

print(list2)

Link nenajdeny: 9
[[0 3]
 [1 6]]


In [22]:
weights = [1] * len(list2)
G = sparse.csr_matrix((weights, (list2[:,0], list2[:,1])), shape=(len(list), len(list)))
rank=pagerank(G, D=0.85, max_iter=10)
rank

array([0.0854701 , 0.0854701 , 0.0854701 , 0.15811962, 0.0854701 ,
       0.0854701 , 0.15811962, 0.0854701 , 0.0854701 , 0.0854701 ])

In [23]:
rank.sum()

1.0